<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/227_Cross_Sell_Upsell_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report generation utilities for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""Report generation utilities for Tier 2 Cross-Sell & Upsell Orchestrator"""

from typing import Dict, List, Any
from datetime import datetime
from pathlib import Path


def format_opportunity_summary(opportunities: List[Dict[str, Any]]) -> str:
    """
    Format a summary of opportunities for the report.

    Args:
        opportunities: List of opportunity dicts

    Returns:
        Formatted markdown string
    """
    if not opportunities:
        return "No opportunities found."

    lines = []
    for i, opp in enumerate(opportunities, 1):
        product_name = opp.get("product_name", "Unknown Product")
        category = opp.get("category", "unknown")
        price = opp.get("price", 0.0)
        final_score = opp.get("final_score", 0.0)
        recommendation_type = opp.get("recommendation_type", "unknown")
        rationale = opp.get("rationale", "")

        lines.append(f"### {i}. {product_name}")
        lines.append(f"")
        lines.append(f"- **Category:** {category}")
        lines.append(f"- **Price:** ${price:.2f}")
        lines.append(f"- **Score:** {final_score:.2f}/10.0")
        lines.append(f"- **Type:** {recommendation_type.replace('_', ' ').title()}")
        lines.append(f"- **Rationale:** {rationale}")
        lines.append(f"")

        # Add score breakdown
        lines.append(f"**Score Breakdown:**")
        lines.append(f"- Business Value: {opp.get('business_value_score', 0):.2f}")
        lines.append(f"- Customer Fit: {opp.get('customer_fit_score', 0):.2f}")
        lines.append(f"- Routine Completeness: {opp.get('routine_completeness_score', 0):.2f}")
        lines.append(f"- Replenishment Urgency: {opp.get('replenishment_urgency_score', 0):.2f}")
        lines.append(f"- Ingredient Match: {opp.get('ingredient_match_score', 0):.2f}")
        lines.append(f"- Upgrade Readiness: {opp.get('upgrade_readiness_score', 0):.2f}")
        lines.append(f"")

    return "\n".join(lines)


def generate_recommendations_report(state: Dict[str, Any]) -> str:
    """
    Generate a markdown report with recommendations.

    Args:
        state: Complete state with customer_data, top_opportunities, opportunity_summary, etc.

    Returns:
        Complete markdown report string
    """
    customer_data = state.get("customer_data", {})
    top_opportunities = state.get("top_opportunities", [])
    opportunity_summary = state.get("opportunity_summary", {})
    customer_routine_steps = state.get("customer_routine_steps", [])
    routine_gaps = state.get("routine_gaps", [])
    routine_step_gaps = state.get("routine_step_gaps", [])

    # Header
    report_lines = [
        "# Tier 2 Cross-Sell & Upsell Recommendations",
        "",
        f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "",
        "---",
        ""
    ]

    # Customer Overview
    customer_name = customer_data.get("name", "Unknown")
    customer_id = state.get("customer_id", "Unknown")
    loyalty_tier = customer_data.get("loyalty_tier", "unknown")
    lifetime_value = customer_data.get("lifetime_value", 0.0)
    skin_concerns = customer_data.get("skin_concerns", [])

    report_lines.extend([
        "## Customer Overview",
        "",
        f"- **Name:** {customer_name}",
        f"- **Customer ID:** {customer_id}",
        f"- **Loyalty Tier:** {loyalty_tier.title()}",
        f"- **Lifetime Value:** ${lifetime_value:.2f}",
        f"- **Skin Concerns:** {', '.join(skin_concerns) if skin_concerns else 'None specified'}",
        "",
        "---",
        ""
    ])

    # Routine Analysis
    report_lines.extend([
        "## Current Routine Analysis",
        "",
        f"- **Routine Steps Completed:** {len(customer_routine_steps)}/5",
        f"- **Steps:** {', '.join(map(str, customer_routine_steps)) if customer_routine_steps else 'None'}",
    ])

    if routine_gaps:
        report_lines.append(f"- **Missing Essential Categories:** {', '.join(routine_gaps)}")
    else:
        report_lines.append("- **Missing Essential Categories:** None (routine is complete)")

    if routine_step_gaps:
        report_lines.append(f"- **Missing Routine Steps:** {', '.join(map(str, routine_step_gaps))}")
    else:
        report_lines.append("- **Missing Routine Steps:** None (all steps covered)")

    report_lines.extend(["", "---", ""])

    # Opportunity Summary
    total_opps = opportunity_summary.get("total_opportunities", 0)
    total_revenue = opportunity_summary.get("total_potential_revenue", 0.0)
    high_value = opportunity_summary.get("high_value_opportunities", 0)
    routine_completeness = opportunity_summary.get("routine_completeness_percent", 0.0)

    report_lines.extend([
        "## Opportunity Summary",
        "",
        f"- **Total Opportunities Found:** {total_opps}",
        f"- **Total Potential Revenue:** ${total_revenue:.2f}",
        f"- **High-Value Opportunities:** {high_value}",
        f"- **Routine Completeness:** {routine_completeness:.1f}%",
        "",
        "### Breakdown by Type",
        "",
        f"- **Cross-Sell Opportunities:** {opportunity_summary.get('total_cross_sell_opportunities', 0)}",
        f"- **Ingredient-Based Opportunities:** {opportunity_summary.get('total_ingredient_opportunities', 0)}",
        f"- **Upgrade Opportunities:** {opportunity_summary.get('total_upgrade_opportunities', 0)}",
        f"- **Replenishment Opportunities:** {opportunity_summary.get('total_replenishment_opportunities', 0)}",
        "",
        "---",
        ""
    ])

    # Top Recommendations
    report_lines.extend([
        "## Top Recommendations",
        "",
        f"The following {len(top_opportunities)} opportunities are recommended based on multi-dimensional scoring:",
        ""
    ])

    if top_opportunities:
        report_lines.append(format_opportunity_summary(top_opportunities))
    else:
        report_lines.append("No top opportunities to display.")

    report_lines.extend(["", "---", ""])

    # Footer
    report_lines.extend([
        "## Notes",
        "",
        "- Scores are calculated using a weighted combination of business value, customer fit, routine completeness, replenishment urgency, ingredient matching, and upgrade readiness.",
        "- Replenishment opportunities are prioritized as they represent immediate customer needs.",
        "- Essential categories (cleanser, moisturizer, SPF) are prioritized for routine completeness.",
        "",
        "---",
        "",
        f"*Report generated by Tier 2 Cross-Sell & Upsell Orchestrator*"
    ])

    return "\n".join(report_lines)


def save_report(report: str, customer_id: str, reports_dir: str) -> str:
    """
    Save report to file.

    Args:
        report: Markdown report string
        customer_id: Customer ID for filename
        reports_dir: Directory to save reports in

    Returns:
        Path to saved report file
    """
    # Create reports directory if it doesn't exist
    reports_path = Path(reports_dir)
    reports_path.mkdir(parents=True, exist_ok=True)

    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"tier2_recommendations_{customer_id}_{timestamp}.md"
    file_path = reports_path / filename

    # Write report
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(report)

    return str(file_path)



# Tests for report generation utilities

In [ ]:
"""Tests for report generation utilities"""

from agents.tier2_crosssell_upsell.utilities.report_generation import (
    format_opportunity_summary,
    generate_recommendations_report,
    save_report
)
from agents.tier2_crosssell_upsell.utilities.data_loading import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)
from agents.tier2_crosssell_upsell.utilities.routine_analysis import (
    analyze_customer_routine
)
from agents.tier2_crosssell_upsell.utilities.opportunity_detection import (
    find_cross_sell_opportunities,
    find_ingredient_based_opportunities
)
from agents.tier2_crosssell_upsell.utilities.scoring import score_opportunity
from pathlib import Path
import tempfile
import os


def test_format_opportunity_summary():
    """Test format_opportunity_summary formats opportunities correctly"""
    print("\n=== Testing format_opportunity_summary ===")

    opportunities = [
        {
            "product_name": "Test Product 1",
            "category": "serum",
            "price": 44.0,
            "final_score": 8.5,
            "recommendation_type": "ingredient_match",
            "rationale": "Matches customer concerns",
            "business_value_score": 7.0,
            "customer_fit_score": 8.0,
            "routine_completeness_score": 5.0,
            "replenishment_urgency_score": 0.0,
            "ingredient_match_score": 10.0,
            "upgrade_readiness_score": 0.0
        }
    ]

    formatted = format_opportunity_summary(opportunities)

    assert isinstance(formatted, str), "Should return a string"
    assert "Test Product 1" in formatted, "Should include product name"
    assert "serum" in formatted, "Should include category"
    assert "$44.00" in formatted, "Should include price"
    assert "8.5" in formatted, "Should include score"

    print("✅ format_opportunity_summary test passed!")
    print(f"   Formatted length: {len(formatted)} characters")

    return formatted


def test_generate_recommendations_report():
    """Test generate_recommendations_report creates complete report"""
    print("\n=== Testing generate_recommendations_report ===")

    # Create a minimal state for testing
    state = {
        "customer_id": "C001",
        "customer_data": {
            "name": "Sarah Lee",
            "loyalty_tier": "silver",
            "lifetime_value": 197.0,
            "skin_concerns": ["dryness", "sensitivity"]
        },
        "top_opportunities": [
            {
                "product_name": "Test Product",
                "category": "serum",
                "price": 44.0,
                "final_score": 8.5,
                "recommendation_type": "ingredient_match",
                "rationale": "Matches customer concerns",
                "business_value_score": 7.0,
                "customer_fit_score": 8.0,
                "routine_completeness_score": 5.0,
                "replenishment_urgency_score": 0.0,
                "ingredient_match_score": 10.0,
                "upgrade_readiness_score": 0.0
            }
        ],
        "opportunity_summary": {
            "total_opportunities": 6,
            "total_potential_revenue": 198.0,
            "high_value_opportunities": 0,
            "routine_completeness_percent": 80.0,
            "total_cross_sell_opportunities": 2,
            "total_ingredient_opportunities": 2,
            "total_upgrade_opportunities": 0,
            "total_replenishment_opportunities": 2
        },
        "customer_routine_steps": [1, 3, 4, 5],
        "routine_gaps": [],
        "routine_step_gaps": [2]
    }

    report = generate_recommendations_report(state)

    assert isinstance(report, str), "Should return a string"
    assert "# Tier 2 Cross-Sell & Upsell Recommendations" in report, "Should have header"
    assert "Customer Overview" in report, "Should have customer overview"
    assert "Sarah Lee" in report, "Should include customer name"
    assert "Current Routine Analysis" in report, "Should have routine analysis"
    assert "Top Recommendations" in report, "Should have recommendations section"
    assert "Test Product" in report, "Should include top opportunity"

    print("✅ generate_recommendations_report test passed!")
    print(f"   Report length: {len(report)} characters")
    print(f"   Report sections: {report.count('##')} main sections")

    return report


def test_save_report():
    """Test save_report saves report to file"""
    print("\n=== Testing save_report ===")

    report = "# Test Report\n\nThis is a test report."
    customer_id = "C001"

    # Use temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        file_path = save_report(report, customer_id, temp_dir)

        assert isinstance(file_path, str), "Should return file path"
        assert Path(file_path).exists(), "File should exist"
        assert file_path.endswith(".md"), "File should be .md"
        assert customer_id in file_path, "File path should include customer_id"

        # Verify content
        with open(file_path, 'r') as f:
            saved_content = f.read()
        assert saved_content == report, "Saved content should match report"

    print("✅ save_report test passed!")
    print(f"   File saved successfully")

    return file_path


def test_report_generation_integration():
    """Test report generation with real data"""
    print("\n=== Testing report generation integration ===")

    # Load real data and run through full pipeline
    from agents.tier2_crosssell_upsell.nodes import (
        goal_node, planning_node, data_loading_node,
        routine_analysis_node, opportunity_detection_node,
        scoring_node, ranking_node
    )

    state = {
        "customer_id": "C001",
        "errors": []
    }

    # Run all nodes
    state = {**state, **goal_node(state)}
    state = {**state, **planning_node(state)}
    state = {**state, **data_loading_node(state)}
    state = {**state, **routine_analysis_node(state)}
    state = {**state, **opportunity_detection_node(state)}
    state = {**state, **scoring_node(state)}
    state = {**state, **ranking_node(state)}

    # Generate report
    report = generate_recommendations_report(state)

    assert isinstance(report, str), "Should return a string"
    assert len(report) > 0, "Report should not be empty"
    assert state["customer_data"]["name"] in report, "Should include customer name"

    # Test saving
    with tempfile.TemporaryDirectory() as temp_dir:
        file_path = save_report(report, state["customer_id"], temp_dir)
        assert Path(file_path).exists(), "Report file should be saved"

    print("✅ Report generation integration test passed!")
    print(f"   Report generated: {len(report)} characters")
    print(f"   Top opportunities in report: {len(state['top_opportunities'])}")

    return report


if __name__ == "__main__":
    """Run all tests"""
    print("=" * 60)
    print("Testing Tier 2 Report Generation Utilities")
    print("=" * 60)

    # Test individual utilities
    test_format_opportunity_summary()
    test_generate_recommendations_report()
    test_save_report()

    # Integration test
    test_report_generation_integration()

    print("\n" + "=" * 60)
    print("✅ All report generation utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_nodes
============================================================
Testing Tier 2 Cross-Sell & Upsell Orchestrator Nodes
============================================================

=== Testing goal_node ===
✅ goal_node test passed!
   Goal: Identify cross-sell and upsell opportunities for Tier 2 customer
   Focus areas: ['routine_completeness', 'ingredient_matching', 'upgrade_opportunities', 'replenishment_needs', 'routine_step_adjacency']

=== Testing goal_node with missing customer_id ===
✅ goal_node error handling test passed!
   Error: goal_node: customer_id is required

=== Testing planning_node ===
✅ planning_node test passed!
   Plan has 6 steps:
      Step 1: data_loading
      Step 2: routine_analysis
      Step 3: opportunity_detection
      Step 4: scoring
      Step 5: ranking
      Step 6: report_generation

=== Testing planning_node with missing goal ===
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

=== Testing goal_node → planning_node integration ===
✅ Integration test passed!
   Customer ID: C001
   Goal: Identify cross-sell and upsell opportunities for Tier 2 customer
   Plan steps: 6

=== Testing data_loading_node ===
✅ data_loading_node test passed!
   Customer: Sarah Lee (C001)
   Products in catalog: 20
   Lookup size: 20

=== Testing data_loading_node with missing customer_id ===
✅ data_loading_node error handling test passed!
   Error: data_loading_node: customer_id is required

=== Testing data_loading_node with customer not found ===
✅ data_loading_node (customer not found) test passed!
   Error: data_loading_node: Customer C999 not found

=== Testing goal_node → planning_node → data_loading_node integration ===
✅ Integration test passed!
   Customer ID: C001
   Customer: Sarah Lee
   Products in catalog: 20
   Plan steps: 6

=== Testing routine_analysis_node ===
✅ routine_analysis_node test passed!
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

=== Testing routine_analysis_node with missing data ===
✅ routine_analysis_node error handling test passed!
   Error: routine_analysis_node: customer_data is required

=== Testing full integration (goal → planning → data_loading → routine_analysis) ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

=== Testing opportunity_detection_node ===
✅ opportunity_detection_node test passed!
   Cross-sell opportunities: 2
   Ingredient opportunities: 2
   Upgrade opportunities: 0
   Replenishment opportunities: 2
   Bundle opportunities: 0

=== Testing opportunity_detection_node with missing data ===
✅ opportunity_detection_node error handling test passed!
   Error: opportunity_detection_node: customer_data is required

=== Testing full integration through opportunity_detection ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Cross-sell: 2
   Ingredient: 2
   Upgrade: 0
   Replenishment: 2
   Total opportunities: 6

=== Testing scoring_node ===
✅ scoring_node test passed!
   Opportunities scored: 6
   Top scored: HydraBalance Prep Toner (score: 4.5)

=== Testing ranking_node ===
✅ ranking_node test passed!
   Ranked opportunities: 6
   Top opportunities: 3
   Total potential revenue: $198.0
   High-value opportunities: 0

=== Testing full integration through ranking ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Total opportunities: 6
   Top opportunities: 3
   Total potential revenue: $198.0
   #1 Recommendation: Barrier Repair Overnight Mask (score: 6.03)

=== Testing report_generation_node ===
✅ report_generation_node test passed!
   Report length: 2541 characters
   Report file: output/tier2_cross_sell_reports/tier2_recommendations_C001_20251124_160742.md

=== Testing complete orchestrator workflow ===
✅ Complete orchestrator workflow test passed!
   Customer: Sarah Lee (C001)
   Total opportunities: 6
   Top recommendations: 3
   Total potential revenue: $198.0
   Report generated: 2541 characters
   Report saved to: output/tier2_cross_sell_reports/tier2_recommendations_C001_20251124_160742.md
   #1 Recommendation: Barrier Repair Overnight Mask (score: 6.03)

============================================================
✅ All tests passed!
============================================================


# Orchestrator

In [ ]:
"""Tier 2 Cross-Sell & Upsell Orchestrator - LangGraph Workflow"""

from langgraph.graph import StateGraph, END
from config import Tier2CrossSellUpsellState
from agents.tier2_crosssell_upsell.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    routine_analysis_node,
    opportunity_detection_node,
    scoring_node,
    ranking_node,
    report_generation_node
)


def create_tier2_orchestrator():
    """
    Create and return the Tier 2 Cross-Sell & Upsell Orchestrator workflow.

    This is a linear workflow following the pattern:
    Goal → Planning → Data Loading → Routine Analysis → Opportunity Detection →
    Scoring → Ranking → Report Generation

    Returns:
        Compiled LangGraph workflow
    """
    # Create StateGraph
    workflow = StateGraph(Tier2CrossSellUpsellState)

    # Add all nodes
    workflow.add_node("goal", goal_node)
    workflow.add_node("planning", planning_node)
    workflow.add_node("data_loading", data_loading_node)
    workflow.add_node("routine_analysis", routine_analysis_node)
    workflow.add_node("opportunity_detection", opportunity_detection_node)
    workflow.add_node("scoring", scoring_node)
    workflow.add_node("ranking", ranking_node)
    workflow.add_node("report_generation", report_generation_node)

    # Set entry point
    workflow.set_entry_point("goal")

    # Linear flow
    workflow.add_edge("goal", "planning")
    workflow.add_edge("planning", "data_loading")
    workflow.add_edge("data_loading", "routine_analysis")
    workflow.add_edge("routine_analysis", "opportunity_detection")
    workflow.add_edge("opportunity_detection", "scoring")
    workflow.add_edge("scoring", "ranking")
    workflow.add_edge("ranking", "report_generation")
    workflow.add_edge("report_generation", END)

    # Compile and return
    return workflow.compile()



# Integration tests for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""Integration tests for Tier 2 Cross-Sell & Upsell Orchestrator"""

from agents.tier2_crosssell_upsell.orchestrator import create_tier2_orchestrator
from config import Tier2CrossSellUpsellState


def test_orchestrator_workflow():
    """Test complete orchestrator workflow via LangGraph"""
    print("\n=== Testing complete orchestrator workflow via LangGraph ===")

    # Create orchestrator
    orchestrator = create_tier2_orchestrator()

    # Initial state
    initial_state: Tier2CrossSellUpsellState = {
        "customer_id": "C001",
        "errors": []
    }

    # Run workflow
    result = orchestrator.invoke(initial_state)

    # Verify final state has all expected fields
    assert "customer_id" in result, "Should have customer_id"
    assert result["customer_id"] == "C001", "Customer ID should match"

    assert "goal" in result, "Should have goal"
    assert "plan" in result, "Should have plan"
    assert "customer_data" in result, "Should have customer_data"
    assert "product_catalog" in result, "Should have product_catalog"
    assert "product_lookup" in result, "Should have product_lookup"
    assert "customer_products" in result, "Should have customer_products"
    assert "routine_gaps" in result, "Should have routine_gaps"
    assert "cross_sell_opportunities" in result, "Should have cross_sell_opportunities"
    assert "scored_opportunities" in result, "Should have scored_opportunities"
    assert "ranked_opportunities" in result, "Should have ranked_opportunities"
    assert "top_opportunities" in result, "Should have top_opportunities"
    assert "opportunity_summary" in result, "Should have opportunity_summary"
    assert "recommendations_report" in result, "Should have recommendations_report"
    assert "report_file_path" in result, "Should have report_file_path"

    # Verify no errors
    errors = result.get("errors", [])
    assert len(errors) == 0, f"Should have no errors, but got: {errors}"

    # Verify report was generated
    report = result["recommendations_report"]
    assert isinstance(report, str), "Report should be a string"
    assert len(report) > 0, "Report should not be empty"
    assert "# Tier 2 Cross-Sell & Upsell Recommendations" in report, "Report should have header"

    # Verify report file exists
    from pathlib import Path
    report_path = Path(result["report_file_path"])
    assert report_path.exists(), "Report file should exist"

    print("✅ Orchestrator workflow test passed!")
    print(f"   Customer: {result['customer_data']['name']} ({result['customer_id']})")
    print(f"   Total opportunities: {result['opportunity_summary']['total_opportunities']}")
    print(f"   Top recommendations: {len(result['top_opportunities'])}")
    print(f"   Total potential revenue: ${result['opportunity_summary']['total_potential_revenue']}")
    print(f"   Report saved to: {result['report_file_path']}")

    if result["top_opportunities"]:
        top = result["top_opportunities"][0]
        print(f"   #1 Recommendation: {top['product_name']} (score: {top['final_score']})")

    return result


def test_orchestrator_with_different_customer():
    """Test orchestrator with a different customer"""
    print("\n=== Testing orchestrator with customer C002 ===")

    orchestrator = create_tier2_orchestrator()

    initial_state: Tier2CrossSellUpsellState = {
        "customer_id": "C002",
        "errors": []
    }

    result = orchestrator.invoke(initial_state)

    assert result["customer_id"] == "C002", "Customer ID should match"
    assert "recommendations_report" in result, "Should have report"
    assert len(result.get("errors", [])) == 0, "Should have no errors"

    print("✅ Orchestrator with C002 test passed!")
    print(f"   Customer: {result['customer_data']['name']} ({result['customer_id']})")
    print(f"   Total opportunities: {result['opportunity_summary']['total_opportunities']}")
    print(f"   Routine gaps: {result['routine_gaps']}")

    return result


if __name__ == "__main__":
    """Run all tests"""
    print("=" * 60)
    print("Testing Tier 2 Cross-Sell & Upsell Orchestrator (LangGraph)")
    print("=" * 60)

    test_orchestrator_workflow()
    test_orchestrator_with_different_customer()

    print("\n" + "=" * 60)
    print("✅ All orchestrator tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_orchestrator
============================================================
Testing Tier 2 Cross-Sell & Upsell Orchestrator (LangGraph)
============================================================

=== Testing complete orchestrator workflow via LangGraph ===
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_030_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)
✅ Orchestrator workflow test passed!
   Customer: Sarah Lee (C001)
   Total opportunities: 6
   Top recommendations: 3
   Total potential revenue: $198.0
   Report saved to: output/tier2_cross_sell_reports/tier2_recommendations_C001_20251124_161308.md
   #1 Recommendation: Barrier Repair Overnight Mask (score: 6.03)

=== Testing orchestrator with customer C002 ===
✅ Orchestrator with C002 test passed!
   Customer: Mark Johnson (C002)
   Total opportunities: 13
   Routine gaps: ['moisturizer', 'spf']

============================================================
✅ All orchestrator tests passed!
============================================================


All tests passed, including the complete LangGraph workflow.

## Orchestrator complete and working

### Final test results
- LangGraph workflow test: PASSED
  - Customer C001: 6 opportunities, $198 potential revenue
  - Top recommendation: Barrier Repair Overnight Mask (score: 6.03)
  - Report generated and saved
- Different customer test: PASSED
  - Customer C002: 13 opportunities
  - Correctly identified routine gaps: ['moisturizer', 'spf']

### What we built

**8 nodes:**
1. `goal_node` — defines analysis goal
2. `planning_node` — creates execution plan
3. `data_loading_node` — loads data
4. `routine_analysis_node` — analyzes routine
5. `opportunity_detection_node` — finds opportunities
6. `scoring_node` — scores opportunities
7. `ranking_node` — ranks and summarizes
8. `report_generation_node` — generates report

**20+ utilities:**
- Data loading (3)
- Routine analysis (3)
- Opportunity detection (4)
- Scoring (7)
- Report generation (3)

**LangGraph workflow:**
- Linear flow wired up
- All nodes connected
- Ready to use

The orchestrator is complete, tested, and ready to use. All tests pass, and it follows orchestrator best practices. The LangSmith warning is a deprecation notice and doesn't affect functionality.



# Tier 2 Cross-Sell & Upsell Recommendations

**Generated:** 2025-11-24 16:13:08

---

## Customer Overview

- **Name:** Mark Johnson
- **Customer ID:** C002
- **Loyalty Tier:** Bronze
- **Lifetime Value:** $93.00
- **Skin Concerns:** acne

---

## Current Routine Analysis

- **Routine Steps Completed:** 3/5
- **Steps:** 1, 2, 3
- **Missing Essential Categories:** moisturizer, spf
- **Missing Routine Steps:** 4, 5

---

## Opportunity Summary

- **Total Opportunities Found:** 13
- **Total Potential Revenue:** $440.00
- **High-Value Opportunities:** 0
- **Routine Completeness:** 60.0%

### Breakdown by Type

- **Cross-Sell Opportunities:** 5
- **Ingredient-Based Opportunities:** 5
- **Upgrade Opportunities:** 0
- **Replenishment Opportunities:** 3

---

## Top Recommendations

The following 3 opportunities are recommended based on multi-dimensional scoring:

### 1. SPF 50 Mattifying Fluid

- **Category:** spf
- **Price:** $37.00
- **Score:** 5.64/10.0
- **Type:** Routine Step Gap
- **Rationale:** Customer missing routine step 5. Completes routine sequence.

**Score Breakdown:**
- Business Value: 6.71
- Customer Fit: 5.29
- Routine Completeness: 10.00
- Replenishment Urgency: 0.00
- Ingredient Match: 10.00
- Upgrade Readiness: 0.00

### 2. BrightFresh Vitamin C Toner

- **Category:** toner
- **Price:** $30.00
- **Score:** 5.36/10.0
- **Type:** Replenishment
- **Rationale:** Product due for replenishment. Replenishment approaching (41 days since purchase, cycle: 45 days).

**Score Breakdown:**
- Business Value: 3.71
- Customer Fit: 6.29
- Routine Completeness: 0.00
- Replenishment Urgency: 14.71
- Ingredient Match: 0.00
- Upgrade Readiness: 0.00

### 3. SPF 50 Mattifying Fluid

- **Category:** spf
- **Price:** $37.00
- **Score:** 5.24/10.0
- **Type:** Ingredient Match
- **Rationale:** Product targets acne concerns which match customer's skin concerns.

**Score Breakdown:**
- Business Value: 6.71
- Customer Fit: 5.29
- Routine Completeness: 8.00
- Replenishment Urgency: 0.00
- Ingredient Match: 10.00
- Upgrade Readiness: 0.00


---

## Notes

- Scores are calculated using a weighted combination of business value, customer fit, routine completeness, replenishment urgency, ingredient matching, and upgrade readiness.
- Replenishment opportunities are prioritized as they represent immediate customer needs.
- Essential categories (cleanser, moisturizer, SPF) are prioritized for routine completeness.

---

*Report generated by Tier 2 Cross-Sell & Upsell Orchestrator*

In [ ]:
        # Deduplicate by product_id - keep the one with highest final_score
        # This handles cases where same product appears in multiple opportunity types
        # When duplicates exist, we combine the recommendation types and rationales
        product_scores = {}
        product_duplicates = {}  # Track all instances for combining

        for scored in scored_opportunities:
            product_id = scored.get("product_id")
            if not product_id:
                continue

            # Track all instances
            if product_id not in product_duplicates:
                product_duplicates[product_id] = []
            product_duplicates[product_id].append(scored)

            # Keep the one with highest score
            if product_id not in product_scores:
                product_scores[product_id] = scored
            else:
                existing_score = product_scores[product_id].get("final_score", 0.0)
                new_score = scored.get("final_score", 0.0)
                if new_score > existing_score:
                    product_scores[product_id] = scored

        # For products with multiple recommendation types, combine them
        for product_id, instances in product_duplicates.items():
            if len(instances) > 1:
                # Get the best one (already in product_scores)
                best = product_scores[product_id]

                # Collect all unique recommendation types and rationales
                all_types = set(inst.get("recommendation_type", "") for inst in instances)
                all_rationales = [inst.get("rationale", "") for inst in instances if inst.get("rationale")]

                # Update with combined information
                if len(all_types) > 1:
                    # Combine types (e.g., "routine_step_gap + ingredient_match")
                    combined_type = " + ".join(sorted(all_types))
                    best["recommendation_type"] = combined_type

                    # Combine rationales
                    unique_rationales = list(dict.fromkeys(all_rationales))  # Preserve order, remove duplicates
                    if len(unique_rationales) > 1:
                        best["rationale"] = " | ".join(unique_rationales)
                    elif unique_rationales:
                        best["rationale"] = unique_rationales[0]

        # Convert back to list
        scored_opportunities = list(product_scores.values())

## Reviewing the report output:

The report looks good overall. One issue: duplicate products in the top recommendations.

What’s working:
- Clear structure: customer overview, routine analysis, opportunity summary, top recommendations
- Useful metrics: routine completeness, potential revenue, breakdown by type
- Score breakdowns: shows how each score was calculated
- Prioritization: replenishment opportunities ranked higher (e.g., C001 #1 is overdue replenishment)

Issue:
- Duplicate products: same product appears multiple times with different recommendation types
  - C001: "HydraBalance Prep Toner" appears as #2 (routine step gap) and #3 (ingredient match)
  - C002: "SPF 50 Mattifying Fluid" appears as #1 (routine step gap) and #3 (ingredient match)

This happens because a product can qualify for multiple types, and we’re not deduplicating before ranking.

Fix: deduplicate by `product_id` before ranking, keeping the highest-scoring instance. Updating the `scoring_node` to deduplicate opportunities:

## Add Deduplication Logic

### What's good about the output

1. Structure: clear sections (Customer Overview, Routine Analysis, Opportunity Summary, Top Recommendations)
2. Metrics: routine completeness, potential revenue, breakdown by type
3. Score transparency: shows all 6 score components
4. Prioritization: replenishment opportunities ranked higher (e.g., C001 #1 is overdue)
5. Actionable: includes rationale for each recommendation

## Issue fixed

- Problem: same product appearing multiple times (e.g., "HydraBalance Prep Toner" as #2 and #3)
- Cause: products can qualify for multiple opportunity types (routine gap + ingredient match)
- Fix: deduplicate by `product_id` before ranking, keeping the highest-scoring instance and combining recommendation types and rationales

## Improvements made

1. Deduplication: one entry per product (highest score wins)
2. Combined types: if a product qualifies for multiple reasons, show "routine_step_gap + ingredient_match"
3. Combined rationales: merge rationales with " | " separator

## Result

- No duplicate products in top recommendations
- Clearer why each product is recommended (combined reasons)
- More accurate opportunity counts



In [ ]:
def scoring_node(state: Tier2CrossSellUpsellState) -> Dict[str, Any]:
    """
    Scoring Node: Score all opportunities multi-dimensionally.

    This node orchestrates scoring using the scoring utilities.
    It scores all opportunities from all types (cross-sell, ingredient, upgrade, replenishment).

    Args:
        state: Current state with all opportunities, customer_data, product_lookup,
               routine_gaps, routine_step_gaps

    Returns:
        Updated state with scored_opportunities (all opportunities with scores)
    """
    errors = state.get("errors", [])

    # Get required data from state
    customer_data = state.get("customer_data")
    product_lookup = state.get("product_lookup")
    routine_gaps = state.get("routine_gaps", [])
    routine_step_gaps = state.get("routine_step_gaps", [])

    # Get all opportunities
    cross_sell = state.get("cross_sell_opportunities", [])
    ingredient = state.get("ingredient_opportunities", [])
    upgrade = state.get("upgrade_opportunities", [])
    replenishment = state.get("replenishment_opportunities", [])
    bundle = state.get("bundle_opportunities", [])

    if not customer_data:
        return {
            "errors": errors + ["scoring_node: customer_data is required"]
        }

    if not product_lookup:
        return {
            "errors": errors + ["scoring_node: product_lookup is required"]
        }

    try:
        # Combine all opportunities
        all_opportunities = cross_sell + ingredient + upgrade + replenishment + bundle

        # Score each opportunity
        scored_opportunities = []
        for opportunity in all_opportunities:
            product_id = opportunity.get("product_id")
            product = product_lookup.get(product_id)

            if not product:
                continue  # Skip if product not found

            # Score the opportunity
            scored = score_opportunity(
                opportunity,
                customer_data,
                routine_gaps,
                routine_step_gaps,
                product
            )
            scored_opportunities.append(scored)

        # Deduplicate by product_id - keep the one with highest final_score
        # This handles cases where same product appears in multiple opportunity types
        # When duplicates exist, we combine the recommendation types and rationales
        product_scores = {}
        product_duplicates = {}  # Track all instances for combining

        for scored in scored_opportunities:
            product_id = scored.get("product_id")
            if not product_id:
                continue

            # Track all instances
            if product_id not in product_duplicates:
                product_duplicates[product_id] = []
            product_duplicates[product_id].append(scored)

            # Keep the one with highest score
            if product_id not in product_scores:
                product_scores[product_id] = scored
            else:
                existing_score = product_scores[product_id].get("final_score", 0.0)
                new_score = scored.get("final_score", 0.0)
                if new_score > existing_score:
                    product_scores[product_id] = scored

        # For products with multiple recommendation types, combine them
        for product_id, instances in product_duplicates.items():
            if len(instances) > 1:
                # Get the best one (already in product_scores)
                best = product_scores[product_id]

                # Collect all unique recommendation types and rationales
                all_types = set(inst.get("recommendation_type", "") for inst in instances)
                all_rationales = [inst.get("rationale", "") for inst in instances if inst.get("rationale")]

                # Update with combined information
                if len(all_types) > 1:
                    # Combine types (e.g., "routine_step_gap + ingredient_match")
                    combined_type = " + ".join(sorted(all_types))
                    best["recommendation_type"] = combined_type

                    # Combine rationales
                    unique_rationales = list(dict.fromkeys(all_rationales))  # Preserve order, remove duplicates
                    if len(unique_rationales) > 1:
                        best["rationale"] = " | ".join(unique_rationales)
                    elif unique_rationales:
                        best["rationale"] = unique_rationales[0]

        # Convert back to list
        scored_opportunities = list(product_scores.values())

    except Exception as e:
        return {
            "errors": errors + [f"scoring_node: Error during scoring: {str(e)}"]
        }

    return {
        "scored_opportunities": scored_opportunities,
        "errors": errors
    }


# Tier 2 Cross-Sell & Upsell Recommendations

**Generated:** 2025-11-24 16:21:15

---

## Customer Overview

- **Name:** Mark Johnson
- **Customer ID:** C002
- **Loyalty Tier:** Bronze
- **Lifetime Value:** $93.00
- **Skin Concerns:** acne

---

## Current Routine Analysis

- **Routine Steps Completed:** 3/5
- **Steps:** 1, 2, 3
- **Missing Essential Categories:** moisturizer, spf
- **Missing Routine Steps:** 4, 5

---

## Opportunity Summary

- **Total Opportunities Found:** 11
- **Total Potential Revenue:** $370.00
- **High-Value Opportunities:** 0
- **Routine Completeness:** 60.0%

### Breakdown by Type

- **Cross-Sell Opportunities:** 5
- **Ingredient-Based Opportunities:** 5
- **Upgrade Opportunities:** 0
- **Replenishment Opportunities:** 3

---

## Top Recommendations

The following 3 opportunities are recommended based on multi-dimensional scoring:

### 1. SPF 50 Mattifying Fluid

- **Category:** spf
- **Price:** $37.00
- **Score:** 5.64/10.0
- **Type:** Ingredient Match + Routine Step Gap
- **Rationale:** Customer missing routine step 5. Completes routine sequence. | Product targets acne concerns which match customer's skin concerns.

**Score Breakdown:**
- Business Value: 6.71
- Customer Fit: 5.29
- Routine Completeness: 10.00
- Replenishment Urgency: 0.00
- Ingredient Match: 10.00
- Upgrade Readiness: 0.00

### 2. BrightFresh Vitamin C Toner

- **Category:** toner
- **Price:** $30.00
- **Score:** 5.36/10.0
- **Type:** Replenishment
- **Rationale:** Product due for replenishment. Replenishment approaching (41 days since purchase, cycle: 45 days).

**Score Breakdown:**
- Business Value: 3.71
- Customer Fit: 6.29
- Routine Completeness: 0.00
- Replenishment Urgency: 14.71
- Ingredient Match: 0.00
- Upgrade Readiness: 0.00

### 3. Oil-Free Clarifying Gel Moisturizer

- **Category:** moisturizer
- **Price:** $33.00
- **Score:** 5.21/10.0
- **Type:** Ingredient Match + Routine Step Gap
- **Rationale:** Customer missing routine step 4. Adjacent to existing steps. | Product targets acne concerns which match customer's skin concerns.

**Score Breakdown:**
- Business Value: 4.14
- Customer Fit: 5.86
- Routine Completeness: 10.00
- Replenishment Urgency: 0.00
- Ingredient Match: 10.00
- Upgrade Readiness: 0.00


---

## Notes

- Scores are calculated using a weighted combination of business value, customer fit, routine completeness, replenishment urgency, ingredient matching, and upgrade readiness.
- Replenishment opportunities are prioritized as they represent immediate customer needs.
- Essential categories (cleanser, moisturizer, SPF) are prioritized for routine completeness.

---

*Report generated by Tier 2 Cross-Sell & Upsell Orchestrator*


The report looks much better. Fixing a bug: replenishment urgency score can exceed 10.0 (e.g., 14.71). Capping it at 10.0:


## Report review

### What's working well

1. No duplicates: top 3 are unique products
   - #1: SPF 50 Mattifying Fluid
   - #2: BrightFresh Vitamin C Toner
   - #3: Oil-Free Clarifying Gel Moisturizer

2. Combined types: shows multiple reasons
   - "Ingredient Match + Routine Step Gap" for #1 and #3
   - Clearer why each product is recommended

3. Combined rationales: merged with " | "
   - #1: "Customer missing routine step 5. Completes routine sequence. | Product targets acne concerns which match customer's skin concerns."
   - Shows both routine gap and ingredient match

4. Accurate counts: 11 opportunities (down from 13 after deduplication)

5. Prioritization: replenishment ranked #2 (high urgency)

### Issue found and fixed

- Bug: Replenishment urgency score exceeded 10.0 (showed 14.71)
- Fix: Added clamping to ensure scores stay within 0-10 range

### Overall assessment

The report is clear and actionable:
- No duplicate products
- Combined recommendation types show multiple reasons
- Score breakdowns are transparent
- Prioritization is correct (replenishment, essential categories)
- Rationales explain why each product is recommended

The orchestrator is production-ready. The fix ensures all scores stay within the 0-10 range.

In [ ]:

Available customers:
  C001: Sarah Lee
  C002: Mark Johnson
  C003: Emily Chen
  C004: Natalie Park
  C005: Alicia Gomez
  C006: Jason Patel
  C007: Omar Hassan
  C008: David Brooks
  C009: Julia Walters
  C010: Kevin Ramirez

(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.run_orchestrator C003

============================================================
Running Tier 2 Cross-Sell & Upsell Orchestrator
Customer ID: C003
============================================================

✅ Orchestrator completed successfully!

📊 Results Summary:
   Customer: Emily Chen (C003)
   Total opportunities: 11
   Top recommendations: 3
   Total potential revenue: $399.0
   Routine completeness: 20.0%

🏆 Top Recommendations:
   1. Radiance Boost Vitamin C Serum
      Score: 6.19/10.0 | Price: $44.00
      Type: replenishment
      Rationale: Product due for replenishment. Replenishment overdue (58 days since purchase, cycle: 30 days).

   2. UltraShield SPF 40
      Score: 5.22/10.0 | Price: $35.00
      Type: ingredient_match + routine_step_gap
      Rationale: Customer missing routine step 5. Completes routine sequence. | Product targets hyperpigmentation concerns which match customer's skin concerns.

   3. Firm+ Overnight Peptide Cream
      Score: 4.24/10.0 | Price: $52.00
      Type: routine_step_gap
      Rationale: Customer missing routine step 4. Adjacent to existing steps.

📄 Report saved to: output/tier2_cross_sell_reports/tier2_recommendations_C003_20251124_162658.md

============================================================


# Tier 2 Cross-Sell & Upsell Recommendations

**Generated:** 2025-11-24 16:26:58

---

## Customer Overview

- **Name:** Emily Chen
- **Customer ID:** C003
- **Loyalty Tier:** Bronze
- **Lifetime Value:** $44.00
- **Skin Concerns:** hyperpigmentation

---

## Current Routine Analysis

- **Routine Steps Completed:** 1/5
- **Steps:** 3
- **Missing Essential Categories:** cleanser, moisturizer, spf
- **Missing Routine Steps:** 1, 2, 4, 5

---

## Opportunity Summary

- **Total Opportunities Found:** 11
- **Total Potential Revenue:** $399.00
- **High-Value Opportunities:** 0
- **Routine Completeness:** 20.0%

### Breakdown by Type

- **Cross-Sell Opportunities:** 9
- **Ingredient-Based Opportunities:** 3
- **Upgrade Opportunities:** 0
- **Replenishment Opportunities:** 1

---

## Top Recommendations

The following 3 opportunities are recommended based on multi-dimensional scoring:

### 1. Radiance Boost Vitamin C Serum

- **Category:** serum
- **Price:** $44.00
- **Score:** 6.19/10.0
- **Type:** Replenishment
- **Rationale:** Product due for replenishment. Replenishment overdue (58 days since purchase, cycle: 30 days).

**Score Breakdown:**
- Business Value: 7.71
- Customer Fit: 4.29
- Routine Completeness: 0.00
- Replenishment Urgency: 10.00
- Ingredient Match: 10.00
- Upgrade Readiness: 0.00

### 2. UltraShield SPF 40

- **Category:** spf
- **Price:** $35.00
- **Score:** 5.22/10.0
- **Type:** Ingredient Match + Routine Step Gap
- **Rationale:** Customer missing routine step 5. Completes routine sequence. | Product targets hyperpigmentation concerns which match customer's skin concerns.

**Score Breakdown:**
- Business Value: 4.43
- Customer Fit: 5.57
- Routine Completeness: 10.00
- Replenishment Urgency: 0.00
- Ingredient Match: 10.00
- Upgrade Readiness: 0.00

### 3. Firm+ Overnight Peptide Cream

- **Category:** moisturizer
- **Price:** $52.00
- **Score:** 4.24/10.0
- **Type:** Routine Step Gap
- **Rationale:** Customer missing routine step 4. Adjacent to existing steps.

**Score Breakdown:**
- Business Value: 8.86
- Customer Fit: 3.14
- Routine Completeness: 10.00
- Replenishment Urgency: 0.00
- Ingredient Match: 0.00
- Upgrade Readiness: 0.00


---

## Notes

- Scores are calculated using a weighted combination of business value, customer fit, routine completeness, replenishment urgency, ingredient matching, and upgrade readiness.
- Replenishment opportunities are prioritized as they represent immediate customer needs.
- Essential categories (cleanser, moisturizer, SPF) are prioritized for routine completeness.

---

*Report generated by Tier 2 Cross-Sell & Upsell Orchestrator*